In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import pickle
import matplotlib.pyplot as pl
import re
import os
from tqdm import tqdm

In [4]:
url_format = 'https://quickstats.censusdata.abs.gov.au/census_services/getproduct/census/2016/quickstat/SSC10098'


# open pickle file
with open('source/source2016.pkl', 'rb') as f:
    source = pickle.load(f)


In [9]:
soup = bs(source['Ashfield'], 'html')
tables = soup.find_all('table')

# Parse Quick Stats from 2001 to 2016

In [14]:
tables[4]

get_quickstat_2016(tables, 'Ashfield').T

,People,Male,Female,Median age,Families,Average children per family for families with children,Average children per family for all families,All private dwellings,Average people per household,Median weekly household income,Median monthly mortgage repayments,Median weekly rent,Average motor vehicles per dwelling
Ashfield,23841.0,0.494,0.506,34.0,5715.0,1.5,0.5,9731.0,2.5,1632.0,2058.0,440.0,1.2


In [34]:
quickstats = {}

for year in ['2001', '2006', '2011', '2016']:
    print('Year={}'.format(year))
   
    # open pickle file
    with open('source{}.pkl'.format(year), 'rb') as f:
        source = pickle.load(f)
    
    # parse quick summary
    stats = []
    for i, (suburb, txt) in enumerate(source.items()):
        if i % 100 == 0:
            print('Quickstat Iteration={}'.format(i))
        soup = bs(txt, 'html')
        # find all tables
        tables = soup.find_all('table')
        # parse quick summary
        try:
            if year == '2016':
                stats.append(get_quickstat_2016(tables, suburb))
            else:
                stats.append(get_quickstat_2011(tables, suburb))
        except ValueError as e:
            print(suburb)
            print(e)
    # integrate all in one table
    quickstats[year] = pd.concat(stats, axis=1).T

Year=2011
Quickstat Iteration=0
Quickstat Iteration=100
Quickstat Iteration=200
Quickstat Iteration=300
Hinchinbrook
Length mismatch: Expected axis has 0 elements, new values have 1 elements
Quickstat Iteration=400
Quickstat Iteration=500
Quickstat Iteration=600
Quickstat Iteration=700
Wickham
Length mismatch: Expected axis has 0 elements, new values have 1 elements
Year=2016
Quickstat Iteration=0
Quickstat Iteration=100
Quickstat Iteration=200
Dural
Length mismatch: Expected axis has 0 elements, new values have 1 elements
Quickstat Iteration=300
Quickstat Iteration=400
Quickstat Iteration=500
Punchbowl
Length mismatch: Expected axis has 0 elements, new values have 1 elements
Quickstat Iteration=600
Quickstat Iteration=700


In [35]:
# Save to disk
for key, df in quickstats.items():
    path = 'Quickstat_{}.csv'.format(key)
    df.to_csv(path)

In [12]:
def get_quickstat_2011(tables, suburn):
    """Get quick stats for each suburb at each year."""
    quickstat = {}
    keys = []
    vals = []

    # parse quick summary
    for people in tables[1: 4]:
        for td in people.find_all('td'):
            if len(td.attrs) > 0 and 'class' in td.attrs and td.attrs['class'] == ['summaryData']:
                vals.append(float(td.text.replace(',', '').replace('$', '')))
            else:
                keys.append(td.text)

    quickstat = {k: [v] for k, v in zip(keys, vals)}
    data = pd.DataFrame.from_dict(quickstat).T
    data.columns = [suburb]
    return data

def get_quickstat_2016(tables, suburb):
    """Get quick stats for each suburb at each year."""
    quickstat = {}
    keys = []
    vals = []

    # parse quick summary
    for people in tables[0: 3]:
        for th, td in zip(people.find_all('th'), people.find_all('td')):
            if td.text == '':
                continue  # average children per family
            value = td.text.replace(',', '').replace('$', '')
            if th.text in ['for families with children', 'for all families']:
                k = 'Average children per family ' + th.text
            else:
                k = th.text
            if '%' in value:
                value = float(value.replace('%', '')) * 0.01
            else:
                value = float(value)
            # append values
            keys.append(k)
            vals.append(value)

    quickstat = {k: [v] for k, v in zip(keys, vals)}
    data = pd.DataFrame.from_dict(quickstat).T
    data.columns = [suburb]
    return data

# Parse Common Table from 2001 to 2016

### Table mapping

In [6]:
common_map = {}
uncommon_map = {}

for year in ['2001', '2006', '2011', '2016']:
    with open('source/source{}.pkl'.format(year), 'rb') as f:
        d = pickle.load(f)
    soup = bs(d['Burwood'], 'html')
    tables = soup.find_all('table')

    cmap = {}
    ucmap = {}
    for i, tb in enumerate(tables):
        try:
            z = re.search(r'(\<\!--).+(-->)', str(tb))
            name = z.group().replace('<!-- ', '').replace('-->', '').strip()
        except AttributeError:
            name = tb.text.strip().split('\n')[0]
        try:
            df = parse_common_table(tb, year)
            if len(df.columns) == 0:
                ucmap[i] = name
            else:
                cmap[i] = name
        except:
            cmap[i] = name
    # append to overall mapping
    common_map[year] = cmap
    uncommon_map[year] = ucmap

In [7]:
non2016 = [0, 1, 2, 3, 4]
yes2016 = [0, 1, 2, 3]
for year in common_map:
    exclusion = yes2016 if year == '2016' else non2016
    common_map[year] = {k: v for k, v in common_map[year].items() if k not in exclusion and v != ''}

# country of birth should be taken out
for year in common_map:
    c = common_map[year]
    topop = None
    for k, v in c.items():
        if v == 'Country of birth':
            uncommon_map[year][k] = v
            topop = k
    common_map[year].pop(k)
uncommon_map

{'2001': {9: 'Religious affiliation, top responses',
  10: 'Language, top responses (other than English)',
  12: 'Occupation',
  13: 'Industry of employment, top responses',
  8: 'Country of birth'},
 '2006': {9: 'Religious affiliation, top responses',
  10: 'Language, top responses (other than English)',
  12: 'Occupation',
  13: 'Industry of employment, top responses',
  21: 'Household composition',
  8: 'Country of birth'},
 '2011': {11: 'Ancestry, top responses',
  14: 'Religious affiliation, top responses',
  15: 'Language, top responses (other than English)',
  19: 'Occupation',
  20: 'Industry of employment, top responses',
  22: 'Travel to work, top responses',
  12: 'Country of birth'},
 '2016': {9: 'Ancestry, top responses',
  12: 'Country of birth of father',
  13: 'Country of birth of mother',
  14: 'Religious affiliation',
  15: 'Language, top responses5',
  18: 'Occupation',
  19: 'Industry of employment',
  21: 'Travel to work',
  10: 'Country of birth'}}

It is consistent that we need to parse table 4 to table 39 for all 4 years.

### Parse common tables

In [8]:
commontables = {}

errortables = None
dataroot = 'census_data'
if not os.path.exists(dataroot):
    os.mkdir(dataroot)
    
for year in ['2001', '2006', '2011', '2016']: 
    # open pickle file
    with open('source/source{}.pkl'.format(year), 'rb') as f:
        source = pickle.load(f)
    
    # iterate all common tables
    for index, tablename in common_map[year].items():
        name = '{}_{}.csv'.format(year, tablename).replace(' ', '_')
        name = name.replace('/', '_')
        path = os.path.join(dataroot, name)
        print('Year={} Table={}'.format(year, tablename))
        if os.path.exists(path):
            continue
        group = []
        for i, (suburb, txt) in enumerate(source.items()):
            if i % 100 == 0:
                print('    Iteration i={}'.format(i))
            soup = bs(txt, 'html')
            # find all tables
            tables = soup.find_all('table')
            try:
                sdata = parse_common_table(tables[index], year)
                group.append(sdata)
            except IndexError:
                print('  Error with {}, index={}'.format(suburb, index))
                errortables = tables
                continue
            
        df = pd.concat(group, axis=0).drop_duplicates()
        print('Saving to', path)
        df.to_csv(path)

Year=2001 Table=Age
Year=2001 Table=Selected characteristics
Year=2001 Table=Registered marital status
Year=2001 Table=Country of birth
Year=2001 Table=Employment
Year=2001 Table=Family type
Year=2001 Table=Dwelling type
Year=2001 Table=Dwelling structure
Year=2001 Table=Tenure
Year=2001 Table=Landlord type
Year=2006 Table=Age
Year=2006 Table=Selected characteristics
Year=2006 Table=Registered marital status
Year=2006 Table=Country of birth
Year=2006 Table=Employment
Year=2006 Table=Median weekly incomes
Year=2006 Table=Family composition
Year=2006 Table=Dwelling structure
Year=2006 Table=Tenure
Year=2006 Table=Landlord type
Year=2011 Table=Age
    Iteration i=0
    Iteration i=100
    Iteration i=200
    Iteration i=300
    Iteration i=400
    Iteration i=500
    Iteration i=600
    Iteration i=700
Saving to census_data/2011_Age.csv
Year=2011 Table=Registered marital status
    Iteration i=0
    Iteration i=100
    Iteration i=200
    Iteration i=300
    Iteration i=400
    Iteration 

In [43]:
errortables[15]
table = errortables[15]
table.find_all('tr')[1].find_all('td')[0]
values = []
for row in table.find_all('tr')[1:]:
    try:
        record = [row.find_all('td')[0].text]
        records = record + [convert(x.text) for x in row.find_all('td')[1:]]
        if len(records) == 1:
            continue
        values.append(records)
    except IndexError:
        print(record)
        print(row.find('td'))

['Total families']
None


In [ ]:
# open pickle file
with open('source{}.pkl'.format(year), 'rb') as f:
    source = pickle.load(f)

In [53]:
with open('source2001.pkl', 'rb') as f:
    d = pickle.load(f)
    
# soup = d['Burwood']
soup = bs(d2016['Burwood'], 'html')
tables = soup.find_all('table')

parse_common_table(tables[16], '2016')

"EmploymentPeople who reported being in the labour force, aged 15 years and over",Worked full-time,Worked part-time,Away from work,Unemployed,Worked full-time (%),Worked part-time (%),Away from work (%),Unemployed (%)
Burwood (NSW),4289,2536,362,795,53.7,31.8,4.5,10.0
New South Wales,2.13452e+06,1.07115e+06,174654,225546,59.2,29.7,4.8,6.3
Australia,6.62306e+06,3.4915e+06,569276,787452,57.7,30.4,5.0,6.9


In [5]:
def convert(val):
    """Convert a value to numeric if it is."""
    val = val.replace(',', '')
    val = float(val) if val.isdigit() else val
    return val

def get_values_2006(table):
    """Parse table for 2006 html page."""
    # find values
    values = []
    for row in table.find_all('tr')[1:]:
        if row.find('td') is None:
            continue
        record = [row.find_all('td')[0].text]
        records = record + [convert(x.text) for x in row.find_all('td')[1:]]
        if len(records) == 1:
            continue
        values.append(records)
    return values

def get_values_2016(table):
    """Parse table for 2016 html page."""
    # find values
    values = []
    for row in table.find_all('tr')[1:]:
        if row.find('td') is None:
            continue
        record = [row.find_all('th')[0].text]
        records = record + [convert(x.text) for x in row.find_all('td')]
        values.append(records)
    return values

def parse_common_table(table, year):
    """Parse table that has common row names across different suburbs."""
    # find column names
    row = table.find_all('tr')[0]
    columns = [x.text for x in row.find_all('th')]
    for i, x in enumerate(columns):
        if x == '%':
            columns[i] = columns[i - 1] + " (%)"
    
    if year != '2016':
        values = get_values_2006(table)
    else:
        values = get_values_2016(table)
    
    data = pd.DataFrame(values, columns=columns).T
    data.columns = data.iloc[0]
    data = data.iloc[1:]

    # create double columns for percentage
    columns = list(data.columns)
    newcolumns = [x + ' (%)' for x in columns]
    for x in newcolumns:
        data[x] = None
        
    # find percentage rows and fill in value
    index = [x for x in data.index if '%' in x]
    for i in index:
        record = data.loc[i]
        rowname = ' '.join(i.split(' ')[:-1])
        for x in columns:
            data.at[rowname, x + ' (%)'] =  record[x]
    
    # drop those rows
    data = data.drop(index)
    return data

In [230]:
ages = tables[4]
# tables[4]

table = tables[4]

# find column names
row = table.find_all('tr')[0]
columns = [x.text for x in row.find_all('th')]
for i, x in enumerate(columns):
    if x == '%':
        columns[i] = columns[i - 1] + " (%)"
columns

values = get_values_2006(table)

values

data = pd.DataFrame(values, columns=columns).T
data.columns = data.iloc[0]
data = data.iloc[1:]

data

Median weekly incomes,Personal,Family,Household,Personal (%),Family (%),Household (%)
Burwood,370,1115,1005,--,--,--
Australia,466,1171,1027,--,--,--


Iteration i=0
Abbotsbury
Iteration i=100
Burwood
Iteration i=200
Dover Heights
Index is out of tables length
Iteration i=300
Heathcote
Index is out of tables length
Index is out of tables length
Iteration i=400
Macquarie Park
Index is out of tables length
Index is out of tables length
Iteration i=500
Oatlands
Index is out of tables length
Iteration i=600
Springfield
Index is out of tables length
Index is out of tables length
Iteration i=700
Whitebridge


In [22]:
suburb = 'Hillsborough'
print(suburb)
soup = bs(txt, 'html')
# find all tables
tables = soup.find_all('table')
# tables[0].find_all('td')[0].text
len(tables)
# sdata = parse_common_table(tables[index], '2016')

Hillsborough


3

In [24]:
pd.concat(group, axis=0).drop_duplicates()

Social marital statusPeople aged 15 years and over,Registered marriage,De facto marriage,Not married,Registered marriage (%),De facto marriage (%),Not married (%)
Abbotsbury,1827,134,1412,54.2,4.0,41.9
New South Wales,2.61263e+06,506133,2.29089e+06,48.3,9.4,42.3
Australia,8.00114e+06,1.75173e+06,7.02497e+06,47.7,10.4,41.9
Abbotsford (NSW),2080,410,1623,50.6,10.0,39.5
Acacia Gardens,1689,133,974,60.4,4.8,34.8
Adamstown,1794,572,2124,40.0,12.7,47.3
Adamstown Heights,2335,273,1329,59.3,6.9,33.8
Airds,408,181,1265,22.0,9.8,68.2
Albion Park,5019,945,3497,53.0,10.0,37.0
Albion Park Rail,2280,555,2147,45.8,11.1,43.1


In [62]:
table = tables[9]
# find column names
row = table.find_all('tr')[0]
columns = [x.text for x in row.find_all('th')]
for i, x in enumerate(columns):
    if x == '%':
        columns[i] = columns[i - 1] + " (%)"
# find values
values = []
for row in table.find_all('tr')[1:]:
    record = [row.find_all('th')[0].text]
    records = record + [convert(x.text) for x in row.find_all('td')]
    values.append(records)

table

<table id="ancestryTable"><!-- Ancestry, top responses -->
<tr>
<th class="firstCol topRow" scope="col"><a class="dictionaryLink" href="/ausstats/abs@.nsf/Lookup/2901.0Chapter20802016" target="_blank" title="A person's ancestry, when used in conjunction with the person's country of birth and whether the person's parents were born in Australia or overseas, provides a good indication of the ethnic background of first and second generation Australians.">Ancestry, top responses</a></th>
<th class="geoCol" scope="col">Burwood (NSW)</th>
<th class="percentCol" scope="col">%</th>
<th class="geoCol supressCol" scope="col">New South Wales</th>
<th class="percentCol supressCol" scope="col">%</th>
<th class="geoCol" scope="col">Australia</th>
<th class="percentCol" scope="col">%</th>
</tr>
</table>

In [126]:
scripts = soup.find_all('script')
scripts[9].attrs

{'type': 'text/javascript'}

In [231]:
javascript = [x for x in scripts if 'src' not in x.attrs and 'type' in x.attrs
              and x.attrs['type'] == 'text/javascript'][0]

# javascript

In [88]:
re.compile("var ancestryData = \[\];").findall(javascript.text)

[]

In [28]:
float(people.find_all('td')[1].text.replace(',', ''))

21260.0